In [1]:
import os
from pprint import pprint
import yaml
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import threading

In [200]:
path = r'D:\Thesis\code\backup\cat-268'
dirlist = os.listdir(path)
hasData = False
fileName = ''
for i in dirlist:
    if i.endswith('.xlsx') and i.startswith('course'):
        hasData = True
        fileName = i
        print("fileName: ",i)
        break
        
if hasData==True:
    result = [] #存放資料
    df = pd.read_excel(path+'\\'+fileName)
    course_id = df['course_id'].tolist()
    course_url = df['course_url'].tolist()
    
    for url in course_url:
        
    
    
    
else:
    print('No data')

fileName:  courses_info_cat_268.xlsx
      Unnamed: 0  course_id  \
0              0     762616   
1              1    1361790   
2              2     937678   
3              3     864146   
4              4    1208634   
...          ...        ...   
9995        9995    4351066   
9996        9996    4215448   
9997        9997    4495526   
9998        9998    4438560   
9999        9999    3047014   

                                            course_name  \
0     The Complete SQL Bootcamp 2022: Go from Zero t...   
1     PMP Exam Prep Seminar - Complete Exam Coverage...   
2     Tableau 2020 A-Z: Hands-On Tableau Training fo...   
3     Agile Crash Course: Agile Project Management; ...   
4     Microsoft Power BI - A Complete Introduction [...   
...                                                 ...   
9995                 How to Gather Project Requirements   
9996  ASQ Certified Master Black Belt (CMBB): Traini...   
9997                                Facilitation Skills   


In [10]:
#讀取yaml檔參數
with open(os.path.expanduser("./config.yaml"), "r") as config:
    cfg = yaml.safe_load(config)

cfg['Udemy']['id']

'nyy28477590@gmail.com'

In [5]:
#設定參數
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
accept = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
accept_encoding = "gzip, deflate, br"
accept_language = "en-US"

opt = webdriver.ChromeOptions()
#opt.add_argument('--user-agent=%s' % user_agent)
#opt.add_argument('--accept=%s' % accept)
#opt.add_argument('--accept-encoding=%s' % accept_encoding)
opt.add_argument('--accept-language=%s' % accept_language)
opt.add_argument('blink-settings=imagesEnabled=false')
opt.add_argument('--enable-javascript')
opt.add_experimental_option("excludeSwitches", ["enable-automation"])
opt.add_experimental_option('useAutomationExtension', False)
opt.add_argument("--disable-blink-features=AutomationControlled")

#匯入Selenium設定
driver = webdriver.Chrome(r'D:\Thesis\code\chromdriver\chromedriver', options=opt)

#固定視窗大小
driver.set_window_size(600, 800)

In [214]:
#登入

#讀取yaml檔參數
with open(os.path.expanduser("./config.yaml"), "r") as config:
    cfg = yaml.safe_load(config)

email = cfg['Udemy']['id']
password = cfg['Udemy']['password']
driver.get('https://www.udemy.com/join/login-popup/')

In [211]:
driver.find_element_by_name("email").send_keys(email)
time.sleep(1)
driver.find_element_by_name("password").send_keys(password)
time.sleep(1)
driver.find_element_by_name("submit").click()

In [5]:
path = r'D:\Thesis\code\backup\cat-332'
dirlist = os.listdir(path)
hasData = False
fileName = ''
for i in dirlist:
    if i.endswith('.xlsx') and i.startswith('course'):
        hasData = True
        fileName = i
        print("fileName: ",i)
        break
        
if hasData==True:
    result = [] #存放資料
    df = pd.read_excel(path+'\\'+fileName)
    course_id = df['course_id'].tolist()
    course_url = df['course_url'].tolist()
    
    for i, url in enumerate(course_url):
        """
        if (i<5000):
            continue
        if (i==6000):
            break
        """
        #先爬100測試
        if (i==100):
            break
        print(str(i+1)+'/'+str(len(course_url)))
        
        course_output = []

        low = 0 #特價
        origin = 0 #原價

        #進入網頁
        driver.get('https://www.udemy.com'+url)

        #存下html
        maxTry = 0
        free = False
        while low==0 and free==False:
            if maxTry>5:
                break
                
            maxTry +=1
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            #取得價格資訊
            if soup.find(class_="slider-menu--price-text-container--28V6G") is None:
                print("None, try again")
                continue
                
            price_info = soup.find(class_="slider-menu--price-text-container--28V6G").find_all('span')

            for p in price_info:
                if p.text.strip().replace(',','').startswith('$') or p.text.strip().replace(',','').startswith('US$'):
                    if low == 0:
                        low = p.text.strip().replace(',','').replace('US','').replace('$','')
                    else:
                        origin = p.text.strip().replace(',','').replace('US','').replace('$','')
                else:
                    if 'Free' in p:
                        free = True
                    continue

            print('low:',low)
            print('origin:',origin)
                    
        if maxTry>5:
            continue
            
        #若只有原價處理
        if origin==0:
            origin = low

        #What you'll learn
        learn = ''
        learn_html = soup.find(class_="component-margin what-you-will-learn--what-will-you-learn--mnJ5T")
        try:
            learn_contents = learn_html.find_all('li')
        except:
            pass
        for l in learn_contents:
            text = l.text.strip()+"\n"
            learn += text

        #MGC
        mgc_html = soup.find(class_="udlite-text-sm component-margin styles--description--3y4KY")
        mgc = " ".join(mgc_html.text.split())

        course_output.append(course_id[i])
        course_output.append(low)
        course_output.append(origin)
        course_output.append(learn)
        course_output.append(mgc)    
        
        result.append(course_output)
    
else:
    print('No data')

fileName:  courses_info_cat_332.xlsx
1/76
2/76
3/76
4/76


KeyboardInterrupt: 

In [6]:
result

[[1361790,
  '390',
  '2290',
  'Earn 35 PDUs/Contact Hours by completing the entire course\nYou will get all the resources you need to pass the PMI PMP certification exam.\nYou will earn 35 exam contact hours as required by PMI\nYou will be able to discuss the PMBOK Guide with confidence.\nExplain the project management processes\nDiscuss the project management knowledge areas\nDemonstrate the formulas, charts, and theories of project management\nCalculate float for complex project network diagrams\nMemorize the formulas for earned value management\nCompare and contrast processes, knowledge areas, theories, and project management best practices\nBuild a strong foundation in Agile project management for the new PMP exam\nComplete hands-on assignments and exercises\n',
  "說明Updated for January 2, 2021, PMP Exam and based on PMI's Exam Content Outline.Join the thousands of others who've completed this top-rated course and passed their PMP exam. You can do this!PMP® Exam Prep Seminar: ID 

In [247]:
count = 0
path = r'D:\Thesis\code\backup'
dirlist = os.listdir(path)
item_count = 0
for i in dirlist:
    sub_path = path+"\\"+i
    sub_dirlist = os.listdir(sub_path)
    for sub in sub_dirlist:
        file_path = sub_path+"\\"+sub
        if file_path.endswith('.xlsx') and sub.startswith('course'):
            df = pd.read_excel(file_path)
            course_id = df['course_id'].tolist()
            item_num = len(course_id)
            print(file_path+":"+str(item_num))
            count += item_num
        else:
            continue
            
print("count:",count)

D:\Thesis\code\backup\cat-268\courses_info_cat_268.xlsx:10000
D:\Thesis\code\backup\cat-269\courses_info_cat_269.xlsx:7306
D:\Thesis\code\backup\cat-273\courses_info_cat_273.xlsx:1861
D:\Thesis\code\backup\cat-274\courses_info_cat_274.xlsx:6196
D:\Thesis\code\backup\cat-276\courses_info_cat_276.xlsx:6480
D:\Thesis\code\backup\cat-278\courses_info_cat_278.xlsx:3315
D:\Thesis\code\backup\cat-283\courses_info_cat_283.xlsx:164
D:\Thesis\code\backup\cat-288\courses_info_cat_288.xlsx:10000
D:\Thesis\code\backup\cat-290\courses_info_cat_290.xlsx:5614
D:\Thesis\code\backup\cat-292\courses_info_cat_292.xlsx:3231
D:\Thesis\code\backup\cat-294\courses_info_cat_294.xlsx:10000
D:\Thesis\code\backup\cat-296\courses_info_cat_296.xlsx:10000
D:\Thesis\code\backup\cat-300\courses_info_cat_300.xlsx:10000
D:\Thesis\code\backup\cat-302\courses_info_cat_302.xlsx:341
D:\Thesis\code\backup\cat-308\courses_info_cat_308.xlsx:354
D:\Thesis\code\backup\cat-310\courses_info_cat_310.xlsx:191
D:\Thesis\code\backup\c

In [237]:
path = r'D:\Thesis\code\backup'
dirlist = os.listdir(path)
item_count = 0
for i in dirlist:
    sub_path = path+"\\"+i
    sub_dirlist = os.listdir(sub_path)
    
    for sub in sub_dirlist:
        file_path = sub_path + "\\" + sub
        print(file_path)
        hasData = False
        fileName = ''
        if sub.endswith('.xlsx') and sub.startswith('course'):
            hasData = True
            fileName = i
            print("fileName: ",i)

        if hasData==True:
            df = pd.read_excel(file_path+'\\'+fileName)
            course_id = df['course_id'].tolist()
            item_num = len(course_id)
            print(fileName+", "+item_num)
            item_count += item_num

        else:
            print('No data')
                
print(item_count)

D:\Thesis\code\backup\cat-267\cat-267-page-1.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-1.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-10.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-100.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-101.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-102.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-103.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-104.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-105.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-106.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-107.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-108.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-109.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-11.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-110.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-111.json
No data
D:\Thesis\code\backup\cat-268\cat-268-page-112

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Thesis\\code\\backup\\cat-268\\courses_info_cat_268.xlsx\\cat-268'

In [185]:
output_df = pd.DataFrame(output)
output_df

,0
0,770
1,4290
2,Achieve learning outcomes for Band 7+ in every...
3,說明IELTS Band 7+ Preparation Course IELTS Cours...


In [225]:
driver.close() # 關閉瀏覽器視窗